<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# Lab 2 - Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%reload_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///bakery.db

'Connected: @bakery.db'

In [12]:
%%sql
SELECT *
FROM used_materials

 * sqlite:///bakery.db
Done.


used_amount,ingredient,product_name,unit
Nut ring,None,450,Flour
Nut ring,None,450,Butter
Nut ring,None,190,Icing sugar
Nut ring,None,225,"Roasted, chopped nuts"
Nut cookie,None,750,Fine-ground nuts
Nut cookie,None,625,"Ground, roasted nuts"
Nut cookie,None,125,Bread crumbs
Nut cookie,None,375,Sugar
Nut cookie,None,350,Egg whites
Nut cookie,None,50,Chocolate


We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [6]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.db
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [8]:
%%sql
SELECT *
FROM movies

 * sqlite:///movies.db
(sqlite3.OperationalError) no such table: movies [SQL: 'SELECT *\nFROM movies'] (Background on this error at: http://sqlalche.me/e/e3q8)


+ Show the performance dates for one of the movies.

In [69]:
%%sql
SELECT screening_date, movie_name
FROM screenings
WHERE movie_name = "The Room"

 * sqlite:///movies.db
Done.


screening_date,movie_name
2019-02-10,The Room


+ Show all data concerning performances at a given theatere
  on a given date.

In [71]:
%%sql
SELECT *
FROM screenings
WHERE theater_name = "Kino" AND screening_date = '2019-02-10'

 * sqlite:///movies.db
Done.


screening_time,screening_date,production_year,movie_name,theater_name
15:00:00,2019-02-10,1994,Forrest Gump,Kino


+ List all customers

In [64]:
%%sql
SELECT full_name, user_name
FROM customers

 * sqlite:///movies.db
Done.


full_name,user_name
Gösta Persson,gosta
Gunnhild Svensson,gunnhild
Göran Olsson,goran
Gustav Olofsson,gurra
Lisa Mattsson,lisa


+ List all tickets

In [65]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.db
Done.


ticket_id,user_name,screening_time,screening_date,theater_name
5df1a8257ad6c3169ab4796a8e22f3d0,gosta,19:00:00,2019-02-10,Filmstaden
dc0845c3da1cf6770991d27afcba92e5,gunnhild,19:00:00,2019-02-10,Filmstaden
0d9edd3da340fc9ed7a134ea4475023a,gurra,19:00:00,2019-02-10,Filmstaden


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [75]:
%%sql
INSERT INTO tickets (user_name, screening_time, screening_date, theater_name)
VALUES    ("lisa", '15:00:00', '2019-02-10', "Kino");

 * sqlite:///movies.db
1 rows affected.
Done.


ticket_id,user_name,screening_time,screening_date,theater_name
b78217ba2a327fdfc116ca0be231158b,gosta,19:00:00,2019-02-10,Filmstaden
0de4e90492a75a68756f810ad448415f,gunnhild,19:00:00,2019-02-10,Filmstaden
651712c88bf99345d81909f9fc1078f9,gurra,19:00:00,2019-02-10,Filmstaden
79ecd45f996c942a7bce22c0c2a0266b,lisa,20:00:00,2019-02-11,Filmstaden
59870eb434d74b71600adb579a84fa6b,lisa,20:00:00,2019-02-11,Filmstaden
9213bbcf7b15c024a7c0dc3b06e1c5db,lisa,15:00:00,2019-02-10,Kino


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2019), we can't do that, instead
  we can the following idea: each row in a SQLite3 table has
  a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries, unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [76]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid();

 * sqlite:///movies.db
Done.


ticket_id
9213bbcf7b15c024a7c0dc3b06e1c5db


+ Try to insert two movie theaters with the same name (this
  should fail).

In [77]:
%%sql
INSERT INTO theaters (theater_name, capacity)
VALUES("Filmstaden", 1234);

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name [SQL: 'INSERT INTO theaters (theater_name, capacity)\nVALUES("Filmstaden", 1234);'] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [78]:
%%sql
INSERT INTO screenings (screening_time, screening_date, production_year, movie_name, theater_name)
VALUES('19:00:00', '2019-03-12', 1999, "Fight Club", "Andreas_bio");

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'INSERT INTO screenings (screening_time, screening_date, production_year, movie_name, theater_name)\nVALUES(\'19:00:00\', \'2019-03-12\', 1999, "Fight Club", "Andreas_bio");'] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist.

In [79]:
%%sql
INSERT INTO tickets (user_name, screening_time, screening_date, theater_name)
VALUES("lisa",'23:00:00', '2019-05-23', "Kino");

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'INSERT INTO tickets (user_name, screening_time, screening_date, theater_name)\nVALUES("lisa",\'23:00:00\', \'2019-05-23\', "Kino");'] (Background on this error at: http://sqlalche.me/e/gkpj)